### Dataset
[Copernicus ER5 land](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land?tab=overview)
### Connexion à l'api 
1. Créer un compte Copernicus : https://cds.climate.copernicus.eu/user/register?destination=%2F%23!%2Fhome
2. Suivre le guide d'installation (https://cds.climate.copernicus.eu/api-how-to) pour créer un fichier .cdsapirc contenant les informations de connexion.
3. Connexion à l'aide de la librairie [cdsapi](https://pypi.org/project/cdsapi/)
### Acceptation des conditions d'utilisation
Si cela n'a pas été depuis le site, à la première exécution, suivre le lien renvoyé par renvoyé par le connecteur pour accepter les conditions d'utilisation du dataset.

In [2]:
%reload_ext autoreload
%autoreload 2
%load_ext jupyter_black

In [14]:
import cdsapi

import matplotlib.path as mplPath
import plotly.express as px
import plotly.graph_objects as go
import json
import numpy as np
from dotenv import load_dotenv
from ecowater_data_research.loaders import load_from_cds

load_dotenv()

True

In [4]:
request = {
    "variable": [
        "total_evaporation",
        "total_precipitation",
    ],
    "year": 2022,
    "month": [1],
    "day": [
        1,
    ],
    "time": "12:00",
    "area": [
        52,
        -5,
        42,
        10,
    ],
    "format": "netcdf",
}
target = "../data/download.nc"
name = "reanalysis-era5-land"

In [5]:
c = cdsapi.Client()
c.retrieve(
    name=name,
    request=request,
    target=target,
)

Result(content_length=63416,content_type=application/x-netcdf,location=https://download-0011-clone.copernicus-climate.eu/cache-compute-0011/cache/data2/adaptor.mars.internal-1677611387.3260286-3174-18-4eacdc63-eae4-4b07-911c-b91266947cb1.nc)

In [6]:
df = load_from_cds(target)

In [7]:
print(df.shape)

(15251, 5)


In [8]:
# df = df[df["time"] == df["time"].unique()[0]]

In [9]:
df

,longitude,latitude,time,e,tp
0,-5.0,52.000000,2022-01-01 12:00:00,-0.000588,0.000591
1,-5.0,51.900002,2022-01-01 12:00:00,-0.000613,0.000768
2,-5.0,51.799999,2022-01-01 12:00:00,-0.000573,0.000744
3,-5.0,51.700001,2022-01-01 12:00:00,-0.000634,0.000542
4,-5.0,51.599998,2022-01-01 12:00:00,NaN,NaN
...,...,...,...,...,...
15246,10.0,42.400002,2022-01-01 12:00:00,NaN,NaN
15247,10.0,42.299999,2022-01-01 12:00:00,NaN,NaN
15248,10.0,42.200001,2022-01-01 12:00:00,NaN,NaN
15249,10.0,42.099998,2022-01-01 12:00:00,NaN,NaN


In [10]:
regions_json = json.load(open("../data/mapping/france_regions.json"))

In [15]:
border = mplPath.Path(
    np.array(regions_json["features"][0]["geometry"]["coordinates"][0])
)

In [23]:
regions_json["features"][1]

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[0.8148247207739, 48.670163564137],
    [0.82767132893029, 48.680720042791],
    [0.85036287028898, 48.682948048025],
    [0.86149783759132, 48.691007914902],
    [0.87671116761022, 48.715496114803],
    [0.88836808739528, 48.718920532577],
    [0.90239998073196, 48.710649225086],
    [0.92119872414556, 48.709175212814],
    [0.95524192597313, 48.716786068922],
    [0.96144420763056, 48.725722137279],
    [0.9772943914531, 48.730138669646],
    [0.98441493173502, 48.725995395669],
    [1.0140639866541, 48.72810030179],
    [1.0206075605327, 48.734789446779],
    [1.0316499412576, 48.730581545643],
    [1.0398026909049, 48.742155339],
    [1.0590712054157, 48.756765184439],
    [1.0686653703661, 48.756913559557],
    [1.0757811283562, 48.749437927852],
    [1.086000043818, 48.756926389494],
    [1.0957264674684, 48.748713222693],
    [1.1126577960236, 48.751863710692],
    [1.1209842348348, 48.767249558527],
    [1.

In [24]:
is_idf = df[["latitude", "longitude"]].apply(
    lambda x: border.contains_point((x["longitude"], x["latitude"])), axis=1
)
df.loc[is_idf, "reg"] = "11"
df.loc[~is_idf, "reg"] = "24"

In [31]:
fig = go.Figure(
    data=go.Scattergeo(
        lat=df.loc[df["reg"] == "11", "latitude"],
        lon=df.loc[df["reg"] == "11", "longitude"],
        marker=dict(
            color=df["tp"],
            opacity=0.7,
            size=2,
        ),
    )
)
# fig.show()

In [27]:
fig = px.choropleth(
    df,
    geojson=regions_json,
    locations="reg",
    color="tp",
    color_continuous_scale="Viridis",
    labels={"tp": "total precipitation"},
)

In [29]:
# fig.show()